<a href="https://colab.research.google.com/github/paulquimbo/nhanes_inferential_2021_23/blob/main/nhanes_inferential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install pyreadstat
!pip install pandas
!pip install requests


In [18]:
import pyreadstat
import pandas
import requests

In [24]:
# Mapping urls
demographics = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/DEMO_L.xpt"
bp = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BPXO_L.xpt"
vitd = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/VID_L.xpt"
hepb = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/HEPB_S_L.xpt"
kidney = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/KIQ_U_L.xpt"
minbehavior = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/PAQ_L.xpt"
weight = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/WHQ_L.xpt"

In [21]:
response = requests.get(bp)
with open("BPXO_L.xpt", "wb") as f:
    f.write(response.content)


In [26]:
df, meta = pyreadstat.read_xport(bp)
print(df.head())

PyreadstatError: File https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BPXO_L.xpt does not exist!